# Execute the process and monitor the execution

To submit an execution request of a deployed process and monitor it, the OGC API Processes API uses the resource highlighted in the table below:

| **Resource**                   | **Path**                                  | **Purpose**                                                                     | **Part**   |
|--------------------------------|-------------------------------------------|---------------------------------------------------------------------------------|------------|
| Landing page                   | `/`                                       | Top-level resource serving as an entry point.                                   | Part 1     |
| Conformance declaration        | `/conformance`                            | Information about the functionality supported by the server.                    | Part 1     |
| API Definition                 | `/api`                                    | Metadata about the API itself.                                                  | Part 1     |
| Process list                   | `/processes`                              | Lists available processes with identifiers and links to descriptions.           | Part 1     |
| Process description            | `/processes/{processID}`                  | Retrieves detailed information about a specific process.                        | Part 1     |
| **Process execution**          | **`/processes/{processID}/execution`**    | Executes a process, creating a job.                                             | **Part 1** |
| Deploy Process                 | `/processes` (POST)                       | Deploys a new process on the server.                                            | Part 2     |
| Replace Process                | `/processes/{processID}` (PUT)            | Replaces an existing process with a new version.                                | Part 2     |
| Undeploy Process               | `/processes/{processID}` (DELETE)         | Removes an existing process from the server.                                    | Part 2     |
| **Job status info**            | `/jobs/{jobID}`                           | Retrieves the current status of a job.                                          | **Part 1** |
| Job results                    | `/jobs/{jobID}/results`                   | Retrieves the results of a job.                                                 | Part 1     |
| Job list                       | `/jobs`                                   | Retrieves a list of submitted jobs.                                             | Part 1     |
| Job deletion                   | `/jobs/{jobID}`                           | Cancels and deletes a job.                                                      | Part 1     |

In [12]:
import requests
import time

namespace = "acme"

ogc_api_endpoint = f"http://zoo-project-dru-service/{namespace}/ogc-api"

Define the input data for the execution:

In [13]:
data = {
    "inputs": {
        "stac_items": [
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"
        ],
        "aoi": "-121.399,39.834,-120.74,40.472",
        "epsg": "EPSG:4326",
        "bands": [
            "green",
            "nir"
        ]
    }
}


Set request headers

In [14]:
headers = {
    "accept": "*/*",
    "Prefer": "respond-async;return=representation",
    "Content-Type": "application/json"
}

In [15]:

process_id = "water-bodies"  # Replace with your process ID if different


# Submit the processing request
response = requests.post(f"{ogc_api_endpoint}/processes/{process_id}/execution", headers=headers, json=data)

# Check if the request was successful
if response.status_code == 201:
    # Parse the response to get the job ID
    job_info = response.json()
    job_id = job_info.get("jobID")
    print(f"Job submitted successfully. Job ID: {job_id}")
    print(f"Monitor job status at: {ogc_api_endpoint}/jobs/{job_id}")
else:
    print(f"Failed to submit job. Status code: {response.status_code}")
    print("Response:", response.text)
    raise ValueError(f"Failed to submit job. Status code: {response.status_code}")


Job submitted successfully. Job ID: e32fc696-8fb9-11ef-a980-8e55bd0a3308
Monitor job status at: http://zoo-project-dru-service/acme/ogc-api/jobs/e32fc696-8fb9-11ef-a980-8e55bd0a3308


Monitor the Job Status

In [16]:
print("\nMonitoring job status...")

job_url = f"{ogc_api_endpoint}/jobs/{job_id}"

headers = {"accept": "application/json"}

while True:
    status_response = requests.get(job_url, headers=headers)
    if status_response.status_code == 200:
        job_status = status_response.json().get("status")
        print(f"Job status: {job_status}")
        
        # Check if the job is completed (either successful or failed)
        if job_status in ["successful", "failed"]:
            break
    else:
        print(f"Failed to get job status. Status code: {status_response.status_code}")
        break
    
    # Wait for a few seconds before checking again
    time.sleep(10)

# Step 4: Retrieve the Job Results if Successful
if job_status == "successful":
    print("\nJob completed successfully. Retrieving results...")
    results_url = f"{ogc_api_endpoint}/jobs/{job_id}/results"
    results_response = requests.get(results_url, headers=headers)
    
    if results_response.status_code == 200:
        results = results_response.json()
        stac_catalog_uri = results.get("stac_catalog", {}).get("value")
        print("STAC Catalog URI:", stac_catalog_uri)
    else:
        print(f"Failed to retrieve job results. Status code: {results_response.status_code}")
else:
    print("\nJob did not complete successfully.")


Monitoring job status...
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: successful

Job completed successfully. Retrieving results...
STAC Catalog URI: {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'stac_version': '1.0.0', 'id': 'S2B_10TFK_20210713_0_L2A', 'properties': {'proj:epsg': 32610, 'proj:geometry': {'type': 'Polygon', 'coordinates': [[[636990.0, 4410550.0], [691590.0, 4410550.0], [691590.0, 4482600.0], [636990.0, 4482600.0], [636990.0, 4410550.0]]]}, 'proj:bbox': [636990.0, 4410550.0, 691590.0, 4482600.0], 'proj:shape': [7205, 5460], 'proj:transform': [10.0, 0.0, 636990.0, 0.0, -10

In [ ]:
results_response.json()